In [2]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [1]:
def coleta_dados(url, zona):
  conteudo = requests.get(url).content
  sopa = BeautifulSoup(conteudo)
  lista_zonas = []
  lista_preços = []
  preços = sopa.find_all('span', class_='price-tag-fraction')
  for elemento in preços:
    p = re.findall('<span class="price-tag-fraction">(.*)</span>', str(elemento))[0]
    p = float(p.replace(".", ""))
    lista_preços.append(p)
    lista_zonas.append(zona)

  sopa2 = BeautifulSoup(conteudo)
  area_e_quartos = sopa2.find_all('ul', class_="ui-search-card-attributes ui-search-item__group__element shops__items-group-details")
  lista_elementos = []
  for elemento in area_e_quartos:
    lista_elementos.append(elemento.text)
  re_quartos = '(.) quarto|quartos'
  re_area = '(.*)m²'
  lista_quartos = []
  lista_area = []
  for el in lista_elementos:
    quarto = re.findall(re_quartos, str(el))
    if quarto != []:
      lista_quartos.append(float(quarto[0]))
    else:
      lista_quartos.append(np.nan)
    area = re.findall(re_area, str(el))
    if area != []:
      a = area[0].replace(".", "").replace(",",".")
      lista_area.append(int(a))
    else:
      lista_area.append(np.nan)


  sopa3 = BeautifulSoup(conteudo)
  tipos = sopa3.find_all('h2', class_="ui-search-item__title ui-search-item__group__element shops__items-group-details shops__item-title")
  lista_tipo = []
  for el in tipos:
    tipo = re.findall('Casa|Apartamento', str(el))
    if len(tipo) > 0:
      lista_tipo.append(tipo[0])
    else:
      lista_tipo.append(np.nan)

  df = pd.DataFrame({'preco': lista_preços, 'quartos': lista_quartos, 'area': lista_area, 'zona': lista_zonas, "tipo": lista_tipo})  
  return df

In [ ]:
paginas = [""] 
num_paginas = 20
for p in range(1, num_paginas):
  paginas.append("_Desde_"+str(1+48*p))

zonas = ['sul', 'norte', 'leste', 'oeste']

dataframes = []

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'
    print("Coletando dados de: ", url)
    df = coleta_dados(url, zona)
    dataframes.append(df)
    sleep(2)

Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_241
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_289
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_337
Coletando dados de:  https://imoveis.mercadolivre.com.br/apartamento

In [ ]:
df = pd.concat(dataframes, ignore_index=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   preço    768 non-null    float64
 1   quartos  767 non-null    float64
 2   area     767 non-null    float64
 3   zona     768 non-null    object 
dtypes: float64(3), object(1)
memory usage: 24.1+ KB


In [ ]:
df.describe

In [ ]:
df.to_csv('dados_ML.csv', index = False)